In [14]:
!pip install -U cohere pinecone-client datasets

In [15]:
key=""

In [16]:
pip install cohere

Note: you may need to restart the kernel to use updated packages.


In [17]:
import cohere
co = cohere.Client("E5rfnQuRu97uHhBQjygny06sOnYQgdK0XxlvGHbq")

In [18]:
from datasets import load_dataset

# load the first 1K rows of the TREC dataset
trec = load_dataset('trec', split='train[:1000]')

Generating train split:   0%|          | 0/5452 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset trec downloaded and prepared to /home/omkar/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2. Subsequent calls will reuse this data.


In [19]:
embeds = co.embed(
    texts=trec['text'],
    model='small',
    truncate='LEFT'
).embeddings


In [21]:
!pip install pinecone-client

In [25]:
import pinecone
pinecone.init(api_key="cb882460-1332-4502-8919-1c8573f69797",environment='asia-southeast1-gcp-free')
index_name = 'cohere-pinecone-trec'

# if the index does not exist, we create it
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=1024,
        metric='cosine'
    )

# connect to index
index = pinecone.Index(index_name)

In [26]:
batch_size = 256

ids = [str(i) for i in range(1000)]
# create list of metadata dictionaries
meta = [{'text': text} for text in trec['text']]

# create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, embeds, meta))

for i in range(0, 1000, batch_size):
    i_end = min(i+batch_size, 1000)
    index.upsert(vectors=to_upsert[i:i_end])

# let's view the index statistics
print(index.describe_index_stats())


{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}


In [32]:
import numpy as np
query = "Irish Tragedies"

# create the query embedding
xq = co.embed(
    texts=[query],
    model='small',
    truncate='LEFT'
).embeddings

print(np.array(xq).shape)

(1, 1024)


In [33]:
res = index.query(xq, top_k=5, include_metadata=True)
for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

0.52: What holidays are celebrated in Ireland ?
0.45: What crop failure caused the Irish Famine ?
0.42: What was the name of the 187s Irish terrorist group in Pennsylvania mining areas ?
0.40: What does NN O `` mean when used as a prefix in Irish surnames ?
0.37: In what city is the headquarters of Sinn Fein ?
